# LLM Response Generation

**Why we're doing this:**
 Take retrieved document chunks and generate coherent answers using a language model.

**What we're doing:**

- Setting up first prototype - done
- Setting up the LLM client (Groq/Llama) - done
- Creating prompt templates for TRL questions - done
- Generating answers from retrieved context - done 

In [2]:
# PERMANENT WORKING IMPORT - USE THIS EVERYWHERE
import sys
import os
import importlib.util

def import_rag_components():
    """Import RAG components using absolute paths - guaranteed to work"""
    current_dir = os.getcwd()
    
    # Import retriever
    retriever_path = os.path.join(current_dir, 'rag_components', 'retriever.py')
    spec = importlib.util.spec_from_file_location("retriever", retriever_path)
    retriever_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(retriever_module)
    
    # Import query_interface  
    query_interface_path = os.path.join(current_dir, 'rag_components', 'query_interface.py')
    spec = importlib.util.spec_from_file_location("query_interface", query_interface_path)
    query_interface_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(query_interface_module)
    
    # Import answer_generator
    answer_generator_path = os.path.join(current_dir, 'rag_components', 'answer_generator.py')
    spec = importlib.util.spec_from_file_location("answer_generator", answer_generator_path)
    answer_generator_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(answer_generator_module)
    
    return (retriever_module.DocumentAwareRetriever, 
            query_interface_module.SimpleQueryInterface,
            answer_generator_module.RAGAnswerGenerator)

# Import the components
DocumentAwareRetriever, SimpleQueryInterface, RAGAnswerGenerator = import_rag_components()
print("🎉 COMPONENTS IMPORTED SUCCESSFULLY!")

# Continue with code
VECTOR_INDEX_PATH = "../04_models/vector_index"
retriever = DocumentAwareRetriever(VECTOR_INDEX_PATH)
query_interface = SimpleQueryInterface(retriever)
answer_generator = RAGAnswerGenerator(query_interface)
print("✅ Generation pipeline ready!")

🎉 COMPONENTS IMPORTED SUCCESSFULLY!
✓ TF-IDF retriever loaded successfully
✓ Template-based RAG answer generator initialized
✅ Generation pipeline ready!


In [14]:
pip install groq

Defaulting to user installation because normal site-packages is not writeable
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [groq]1/2 [groq]
Note: you may need to restart the kernel to use updated packages.


In [19]:
# CELL: LLM Client Setup
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Groq client
def setup_groq_client():
    """Set up and return Groq client with error handling"""
    api_key = os.getenv('GROQ_API_KEY')
    
    if not api_key:
        raise ValueError("❌ GROQ_API_KEY not found in environment variables")
    
    client = Groq(api_key=api_key)
    print("✅ Groq client initialized successfully")
    return client

# Test the client
try:
    groq_client = setup_groq_client()
    print("🎉 LLM client ready for integration!")
except Exception as e:
    print(f"❌ Failed to initialize LLM client: {e}")

✅ Groq client initialized successfully
🎉 LLM client ready for integration!


In [23]:
# CELL: Test LLM Connection
# Why: Verify Groq API works and model responds correctly
# What: Send simple test query to confirm setup is functional
def test_llm_connection():
    try:
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",  # Fast, free model for testing
            messages=[{"role": "user", "content": "Reply only with 'API connected'"}],
            max_tokens=10,
            temperature=0.1
        )
        print(f"✅ LLM Connected: {response.choices[0].message.content}")
        return True
    except Exception as e:
        print(f"❌ LLM Failed: {e}")
        return False

test_llm_connection()

✅ LLM Connected: API connected


True

In [24]:
# CELL: Integrate with Your Generator
def generate_with_llm(query, context):
    """Generate answer using Groq/Llama"""
    prompt = f"""
    Based on the following context, answer the user's question.
    
    Context: {context}
    
    Question: {query}
    
    Answer:
    """
    
    response = groq_client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=500,
        temperature=0.3
    )
    
    return response.choices[0].message.content

print("🚀 LLM integration code ready!")

🚀 LLM integration code ready!


In [25]:
# CELL: TRL-Specific Prompt Template
# Why: Provide structured guidance for technology maturity assessment
# What: Template that incorporates TRL definitions for transition analysis

TRL_TRANSITION_TEMPLATE = """
CONTEXT:
{context}

TRL DEFINITIONS:
- TRL 1-2: Basic research, scientific formulation
- TRL 3-4: Experimental proof of concept, lab validation  
- TRL 5-6: Technology demonstration, prototype testing
- TRL 7-8: System prototype, operational environment
- TRL 9: Actual system proven, commercial deployment

QUESTION: {question}

ANALYSIS REQUEST:
Based on the context and TRL definitions above, analyze which research topics are transitioning from academic research (TRL 1-4) to commercial application (TRL 5-9). For each identified technology:

1. Current estimated TRL level
2. Evidence of transition/commercialization
3. Key players or organizations involved
4. Timeline indicators for maturity

ANSWER:
"""

PROMPT_TEMPLATES = {
    "technical": """
Context: {context}
Question: {question}
Answer based on the context. If unsure, say so. Cite sources.
Answer:""",
    
    "trl_transition": TRL_TRANSITION_TEMPLATE,
    
    "trends": """
Context: {context}
What are the trends in: {question}
Identify 3-5 key trends with evidence:
Answer:"""
}

def get_prompt_template(question):
    """Select appropriate template based on question content"""
    if any(keyword in question.lower() for keyword in ['trl', 'mature', 'transition', 'academy to application', 'commercial']):
        return PROMPT_TEMPLATES["trl_transition"]
    elif any(keyword in question.lower() for keyword in ['trend', 'growing', 'latest']):
        return PROMPT_TEMPLATES["trends"]
    else:
        return PROMPT_TEMPLATES["technical"]

print("✅ TRL-specific prompt template added for maturity analysis")

✅ TRL-specific prompt template added for maturity analysis


In [26]:
# CELL: Optimized Prompt Templates
# Why: Provide specialized prompts only when TRL context is relevant
# What: Keep templates clean and focused on actual query needs

PROMPT_TEMPLATES = {
    "descriptive": """
CONTEXT: {context}

QUESTION: {question}

INSTRUCTIONS:
- Provide a comprehensive overview based on the context
- List specific examples and entities mentioned
- Cite sources for each key point
- If context is insufficient, acknowledge limitations

ANSWER:""",
    
    "explanatory": """
CONTEXT: {context}

QUESTION: {question}

INSTRUCTIONS:
- Provide detailed explanations with examples
- Break down complex concepts into understandable parts
- Show relationships between different elements
- Cite specific sources that support your explanation

ANSWER:""",
    
    "trends": """
CONTEXT: {context}

QUESTION: {question}

INSTRUCTIONS:
- Identify 3-5 key trends with supporting evidence
- Note the velocity and direction of each trend
- Mention any disruptive or accelerating factors
- Provide timeline context where available
- Cite sources for each trend identified

ANSWER:""",
    
    "forecast": """
CONTEXT: {context}

QUESTION: {question}

INSTRUCTIONS:
- Analyze timing and maturity indicators
- Distinguish between near-term (1-2 years) and longer-term developments
- Note any dependencies or barriers to adoption
- Provide confidence levels for predictions
- Cite evidence supporting timeline estimates

ANSWER:""",
    
    # TRL template ONLY for queries that explicitly need it
    "trl_transition": """
CONTEXT: {context}

QUESTION: {question}

INSTRUCTIONS:
- Assess technology maturity levels based on evidence in context
- Identify which technologies are transitioning from research to application
- Note key organizations driving commercialization
- Estimate development timelines where evidence exists
- Provide specific citations for maturity assessments

ANSWER:"""
}

def get_prompt_template(question):
    """Select appropriate template - TRL only for explicit maturity questions"""
    question_lower = question.lower()
    
    # TRL definitions ONLY for queries 8-9 that explicitly need maturity assessment
    if any(keyword in question_lower for keyword in ['trl', 'mature', 'transition', 'academy to application', 'commercial', 'moving from academy']):
        return PROMPT_TEMPLATES["trl_transition"]
    
    # Forecast and timing questions (query 6)
    elif any(keyword in question_lower for keyword in ['next year', 'timing', 'forecast', 'likely to mature', 'when will']):
        return PROMPT_TEMPLATES["forecast"]
    
    # Trend and velocity questions (queries 3,7)
    elif any(keyword in question_lower for keyword in ['trend', 'growing', 'latest', 'velocity', 'fastest']):
        return PROMPT_TEMPLATES["trends"]
    
    # Explanatory questions (queries 2,4)
    elif any(keyword in question_lower for keyword in ['summarize', 'explain', 'pain points', 'use cases', 'how is']):
        return PROMPT_TEMPLATES["explanatory"]
    
    # Default for descriptive/factual questions (queries 1,5)
    else:
        return PROMPT_TEMPLATES["descriptive"]

print("✅ Optimized prompt templates - TRL only for relevant queries")
print("✅ Queries 1-7 use clean templates, 8-9 get TRL context when needed")

✅ Optimized prompt templates - TRL only for relevant queries
✅ Queries 1-7 use clean templates, 8-9 get TRL context when needed


# Response Quality Setup

**Why we're doing this:** 
Ensure answers are relevant and properly cite sources.

**What we're doing:**

- Formatting retrieved chunks as context - to-do
- Adding source citations to responses - to-do
- Setting temperature parameters for consistency - to-do

**Files accessed:**
/models/vector_index/ (FAISS index)
/data/processed/chunked_corpus.json (for source metadata)